## Import

In [1]:
import pandas as pd
import numpy as np
import random
import pickle
import spacy
from spacy.training import Example
import warnings;warnings.filterwarnings('ignore')

## Read data

In [2]:
satur_raw = pd.read_csv('../data/satur/satur_20240313.csv', usecols=[1], names=['pro_nm'], encoding='utf-8')\
            .drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'data_id'})
satur_product = pd.read_csv('../data/satur/satur_product.csv', usecols=['enc_nm','kor_nm'], encoding='cp949')\
            .drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'product_id'})

## Data Cleansing

In [3]:
# 불필요한 특수문자를 제거한다.
# [참고] MassAdoption\3월\240313_SaturAssociationRule\1. DataCleansing\3. Cleansing_ProductTable.ipynb
satur_raw['DC_pro_nm'] = satur_raw.pro_nm.apply(lambda x: x[x.rindex('>')+1:] if '<' in x else x)\
                        .apply(lambda x: x[x.rindex(')')+1:] if '(' in x else x)\
                        .apply(lambda x: x[x.rindex(']')+1:] if '[' in x else x)

In [4]:
satur_product['DC_enc_nm'] = satur_product['enc_nm']\
                             .apply(lambda x: x[x.rindex('>')+1:] if '<' in x else x)\
                             .apply(lambda x: x[x.rindex(')')+1:] if '(' in x else x)\
                             .apply(lambda x: x[x.rindex(']')+1:] if '[' in x else x).str.replace('\n','').str.strip()

In [5]:
satur_product['DC_kor_nm'] = satur_product['kor_nm'].fillna('-')\
                             .apply(lambda x: x[x.rindex('>')+1:] if '<' in x else x)\
                             .apply(lambda x: x[x.rindex(')')+1:] if '(' in x else x)\
                             .apply(lambda x: x[x.rindex(']')+1:] if '[' in x else x).str.replace('\n','').str.strip()

## Split color info

In [6]:
getColor = satur_product[satur_product.enc_nm.str.contains('-')]

In [7]:
getColor['Color_enc_nm'] = getColor.DC_enc_nm.str[::-1].str.split(' - ', expand=True)[0].str[::-1]
getColor['Product_enc_nm'] = getColor[['DC_enc_nm','Color_enc_nm']]\
                             .apply(lambda x: x[0][:x[0].index(x[1])-2] if x[1] in x[0] else x[0], axis=1)
# 영어 상품명
TRAIN_ENC_DATA = [(i, {"entities":[(i.rindex('-')+2,len(i), "COLOR")]}) for i in getColor.DC_enc_nm.values]

In [8]:
n_colors = getColor.Color_enc_nm.str.count(' ')+1
getColor['Color_kor_nm'] = [' '.join(COLOR.split()[-N:]) for COLOR, N in list(zip(getColor.DC_kor_nm, n_colors)) if N]
getColor['Product_kor_nm'] = getColor[['DC_kor_nm','Color_kor_nm']]\
                             .apply(lambda x: x[0][:x[0].index(x[1])-1] if x[1] in x[0] else x[0], axis=1)

## spaCy NER

In [9]:
# 영어 모델 학습
enc_nlp = spacy.blank("en")
if "ner" not in enc_nlp.pipe_names:
    ner = enc_nlp.create_pipe("ner")
    enc_nlp.add_pipe("ner")
else:
    ner = enc_nlp.get_pipe("ner")

# 'COLOR' 엔티티 추가
ner.add_label("COLOR")

# 모델 훈련
optimizer = enc_nlp.begin_training()
for itn in range(6):  # 6번의 반복 훈련
    random.shuffle(TRAIN_ENC_DATA)
    losses = {}
    for text, annotations in TRAIN_ENC_DATA:
        doc = enc_nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        enc_nlp.update([example], drop=0.5, losses=losses)
    print(losses)

{'ner': 405.9557896050791}
{'ner': 93.20782349693889}
{'ner': 40.44284183418084}
{'ner': 42.33473774070164}
{'ner': 20.663546943535202}
{'ner': 18.85016809055798}


In [ ]:
'''# 한글 모델 학습
# mecab-ko, mecab-ko-dic, natto 설치가 필요하다. 작동하지 않을 경우 enc_nm color의 글자로 유추해 변환한다.
kor_nlp = spacy.blank("ko")
if "ner" not in kor_nlp.pipe_names:
    ner = kor_nlp.create_pipe("ner")
    kor_nlp.add_pipe("ner")
else:
    ner = kor_nlp.get_pipe("ner")

# 'COLOR' 엔티티 추가
ner.add_label("COLOR")

# 모델 훈련
optimizer = kor_nlp.begin_training()
for itn in range(6):  # 6번의 반복 훈련
    random.shuffle(TRAIN_KOR_DATA)
    losses = {}
    for text, annotations in TRAIN_KOR_DATA:
        doc = kor_nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        kor_nlp.update([example], drop=0.5, losses=losses)
    print(losses)'''

In [ ]:
import spacy
from spacy.tokens import Doc
from konlpy.tag import Okt

class KoreanTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab
        self.tokenizer = Okt()

    def __call__(self, text):
        words = self.tokenizer.morphs(text)
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)

# 빈 모델 생성 및 한국어 토크나이저 설정
nlp = spacy.blank("ko")
nlp.tokenizer = KoreanTokenizer(nlp.vocab)

# NER 파이프라인 추가 (필요한 경우)
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe("ner", last=True)

# 모델 사용 예시
doc = nlp("한국어 문장을 여기에 입력하세요.")
for token in doc:
    print(token.text)

In [ ]:
import spacy

# 다국어 모델 로드
nlp = spacy.load("xx_ent_wiki_sm")

# 모델 사용 예시
doc = nlp("이 문장에는 서울, 뉴욕, 파리 같은 지명이 포함되어 있습니다.")
for ent in doc.ents:
    print(ent.text, ent.label_)

In [10]:
# 색상 규칙에 부합하지 않는 데이터에 자연어 처리를 한다.
# enc_nlp = pickle.load(open('../data/satur/color_ner_model.pkl','rb'))
unrule = satur_product.loc[[i for i in satur_product.index if i not in getColor.index]]

unrule['Color_enc_nm'] = [enc_nlp(i).ents for i in unrule.DC_enc_nm]
# unrule['Color_enc_nm'].apply(lambda x: x[0].text if len(x)!= 0 else '-')
unrule['Color_enc_nm'] = ['French Brown','-','-','-','Black','Ivory',
                          '-','Melange Ivory','-','-','Summer Forest',
                          'Navy','Ivory','Saddle Ivory','-','-','Ivory',
                          'Oyster Gray','-','-','-','-','-','-']
# 오류 데이터 제거
unrule.drop([344], inplace=True)

In [11]:
# 영어 색상명 병합, 영어 상품명 생성
unrule['Product_enc_nm'] = unrule[['DC_enc_nm','Color_enc_nm']]\
                           .apply(lambda x: x[0] if x[1] not in x[0] else x[0][:x[0].index(x[1])-1], axis=1)
unrule.loc[[648,686], 'Product_enc_nm'] = ''

In [12]:
# 한글 색상명 병합(글자 수), 한글 상품명 생성
unrule['Color_kor_nm'] = ['프렌치 브라운','-','-','-','블랙','아이보리',
                          '-','-','-','썸머 포레스',
                          '네이비','아이보리','새들 아이보리','-','-','아이보리',
                          '오이스터 그레이','-','-','-','-','-','-']
unrule['Product_kor_nm'] = unrule[['DC_kor_nm','Color_kor_nm']]\
                           .apply(lambda x: x[0] if x[1] not in x[0] else x[0][:x[0].index(x[1])-1], axis=1)

In [13]:
DC_satur_product = pd.concat([getColor, unrule])

## Save data

In [16]:
# COLOR NER 모델을 저장한다.
# pickle.dump((enc_nlp), open('../data/satur/color_ner_model.pkl','wb'))

In [21]:
DC_satur_product.to_csv('../data/satur/DC_satur_product.csv', encoding='cp949', index=False)